# 9-9. 프로젝트 - Movielens 영화 추천 실습

이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

cloud jupyter 를 열고 aiffel/recommendata_iu/ 디렉토리에 python3로 ipynb파일을 생성 해 주세요 jupyter notebook 환경에서 프로젝트 코드를 작성하고 제출하시면 됩니다.

이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
별점을 시청횟수로 해석해서 생각하겠습니다.
또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
Cloud Storage에 미리 업로드 된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결 해 줍니다.

Cloud shell에서 아래 명령어를 입력해주세요.

```
mkdir -p ~/aiffel/recommendata_iu/data/ml-1m
ln -s ~/data/ml-1m/* ~/aiffel/recommendata_iu/data/ml-1m
```

## 1) 데이터 준비와 전처리
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [1]:
import pandas as pd
import os

from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

In [2]:
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [5]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

## 2) 분석해 봅시다.

* ratings에 있는 유니크한 영화 개수

* rating에 있는 유니크한 사용자 수

* 가장 인기 있는 영화 30개(인기순)

In [7]:
# 고유한영화 를 찾아내는 코드
movies_unique = movies['title'].unique()

# 영화명을 코드로 idx는 index
title_to_idx = {k:v for k,v in enumerate(movies_unique)}

In [8]:
# ratings에 있는 유니크한 영화 개수
print(ratings['movie_id'].nunique())

# rating에 있는 유니크한 사용자 수
print(ratings['user_id'].nunique())

# 인기 많은 영화 30개
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

3628
6039


movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [9]:
ratings.head(15)
# 선호영화 ID : 1035, 2918, 2398, 938, 595

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [10]:
print(ratings['user_id'].min())
print(ratings['user_id'].max())
# user_id : 7777 생성

1
6040


In [11]:
# 선호하는 영화 5가지
my_favorite = ['1035', '2918', '2398', '938', '595']

# '7777'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['7777']*5, 'movie_id': my_favorite, 'count':[5]*5,'timestamp':[0]*5})

if not ratings.isin({'user_id':['7777']})['user_id'].any():
    ratings = ratings.append(my_playlist)

# 잘 추가되었는지 확인
ratings.tail

<bound method NDFrame.tail of    user_id movie_id  count  timestamp
0        1     1193      5  978300760
1        1      661      3  978302109
2        1      914      3  978301968
3        1     3408      4  978300275
4        1     2355      5  978824291
..     ...      ...    ...        ...
0     7777     1035      5          0
1     7777     2918      5          0
2     7777     2398      5          0
3     7777      938      5          0
4     7777      595      5          0

[836483 rows x 4 columns]>

## 4) CSR matrix를 직접 만들어 봅시다.

In [12]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique  = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx  = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

print(user_to_idx['7777'])    # 유저 중 마지막으로 추가된 유저
print(movie_to_idx[1193])     # 첫번째 데이터 확인

6039
0


In [13]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # ratings['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie_id 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

ratings

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,count,timestamp
0,0,0,5,978300760
1,0,1,3,978302109
2,0,2,3,978301968
3,0,3,4,978300275
4,0,4,5,978824291
...,...,...,...,...
0,6039,3628,5,0
1,6039,3629,5,0
2,6039,3630,5,0
3,6039,3631,5,0


In [14]:
# CSR METRIX 생성
from scipy.sparse import csr_matrix

num_user  = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3633 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [15]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [16]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [17]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3633x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [18]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [19]:
my_id, movie_1035_id = user_to_idx['7777'], movie_to_idx['1035']
my_vector, my_movie_vector = als_model.user_factors[my_id], als_model.item_factors[movie_1035_id]

my_vector

array([-0.00840781,  0.0113592 , -0.00831262, -0.00463314,  0.01331623,
       -0.01999337, -0.02603151,  0.0113066 , -0.02349801, -0.00270255,
       -0.0218832 ,  0.01007402,  0.01062614, -0.00862445,  0.01070968,
        0.01763801,  0.01397138, -0.02426327, -0.00245043, -0.00465738,
        0.0267834 ,  0.02174296, -0.01299721, -0.01129712,  0.03593904,
        0.0033514 , -0.02132326, -0.01497498,  0.01338178, -0.01961353,
        0.01415914, -0.00314916,  0.00322307, -0.00288308,  0.01580591,
        0.00494699,  0.01795966,  0.01537743,  0.00840407, -0.0193224 ,
        0.01102238, -0.01537501,  0.01300465, -0.00542041,  0.01369915,
       -0.0123802 , -0.00394402, -0.00965467,  0.02036744, -0.01035865,
       -0.023769  ,  0.00244144,  0.02595178, -0.02544961, -0.02257939,
       -0.02653877, -0.02001136,  0.00716687, -0.00049212,  0.00236762,
        0.00439306,  0.02437705,  0.02014806,  0.02894879,  0.00984494,
        0.00141597, -0.03029048, -0.00615879,  0.00676321,  0.02

In [20]:
my_movie_vector

array([ 1.15107599e-04,  2.91579054e-04,  5.67480340e-04,  3.95541196e-04,
        6.13309385e-04,  1.87278580e-04, -1.49131272e-04,  3.38481710e-04,
        1.75151581e-04,  2.18853937e-04, -2.84329362e-05,  4.00596298e-04,
        5.46211086e-04,  3.88394896e-04,  6.18088932e-04,  5.16965636e-04,
        6.92062546e-04, -1.57628179e-04,  2.36682550e-04,  2.91122647e-04,
        7.59043614e-04,  7.17710238e-04,  2.26070188e-04,  2.63076508e-04,
        1.00297900e-03,  7.14300375e-04, -1.21733654e-04,  7.99616028e-05,
        8.27414216e-04, -6.69605433e-05,  4.41933691e-04,  1.80796647e-04,
        5.42310125e-04,  3.10692412e-04,  7.56737485e-04,  5.85029833e-04,
        6.01143227e-04,  5.80779684e-04,  4.02765523e-04, -2.66100775e-04,
        3.13960918e-04,  2.00813447e-04,  7.31599226e-04,  3.46750749e-04,
        6.09471695e-04, -1.09307730e-04,  2.58866436e-04,  2.71238416e-04,
        5.89682022e-04,  9.86651939e-05,  1.17961667e-04,  2.95081874e-04,
        1.13318383e-03,  

In [21]:
# 내가 선호하는 5가지 영화 중 하나의 선호도 파악
np.dot(my_vector, my_movie_vector)

0.00044757547

In [22]:
# 내가 선호하는 않은 영화의 선호도 파악
# id: 1 -> Toy Story (1995)
toy_story  = movie_to_idx[1]
toy_story_vector = als_model.item_factors[toy_story]
np.dot(my_vector, toy_story_vector)

0.00055249914

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [23]:
# 내가 좋아하는 영화 찾기 : 타이타닉 찾기 ( id : 1721 )
movies[movies['title'].str.contains('Tita')]

,movie_id,title,genre
1672,1721,Titanic (1997),Drama|Romance
2088,2157,"Chambermaid on the Titanic, The (1998)",Romance
3334,3403,Raise the Titanic (1980),Drama|Thriller
3335,3404,Titanic (1953),Action|Drama
3676,3745,Titan A.E. (2000),Adventure|Animation|Sci-Fi
3846,3916,Remember the Titans (2000),Drama


In [24]:
# 타이타닉[1721]과 비슷한 영화 추천
titanic = movie_to_idx[1721]
similar_movie = als_model.similar_items(titanic, N=15)
similar_movie

[(27, 1.0),
 (384, 0.5305401),
 (3493, 0.42016423),
 (641, 0.4168241),
 (336, 0.40818116),
 (39, 0.40736434),
 (3569, 0.4044995),
 (385, 0.4000131),
 (1352, 0.39931893),
 (426, 0.39724517),
 (498, 0.3918919),
 (1668, 0.38183677),
 (2717, 0.381185),
 (626, 0.38013047),
 (666, 0.37247863)]

In [25]:
# 타이타닉과 비슷한 영화 목록( 정말인가.. ? )
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[title_to_idx[i[0]] for i in similar_movie]

['Persuasion (1995)',
 'Boys Life (1995)',
 'Committed (2000)',
 'Courage Under Fire (1996)',
 'War, The (1994)',
 'Cry, the Beloved Country (1995)',
 'Moonraker (1979)',
 'Colonel Chabert, Le (1994)',
 'Star Trek V: The Final Frontier (1989)',
 'Calendar Girl (1993)',
 'Next Karate Kid, The (1994)',
 'Scream 2 (1997)',
 'Haunted Honeymoon (1986)',
 'All Dogs Go to Heaven 2 (1996)',
 'Tarantella (1995)']

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [26]:
def get_similar_movie(movie_id: str):
    movie_id = movie_to_idx[movie_id]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [title_to_idx[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [27]:
# 내가 좋아할만한 영화 추천 : 슈퍼맨 ( id : 2640 )
movies[movies['title'].str.contains('Superman')]


,movie_id,title,genre
2571,2640,Superman (1978),Action|Adventure|Sci-Fi
2572,2641,Superman II (1980),Action|Adventure|Sci-Fi
2573,2642,Superman III (1983),Action|Adventure|Sci-Fi
2574,2643,Superman IV: The Quest for Peace (1987),Action|Adventure|Sci-Fi


In [28]:
# 추천 결과
get_similar_movie(2640)

['Second Best (1994)',
 'Touki Bouki (Journey of the Hyena) (1973)',
 'Brady Bunch Movie, The (1995)',
 'Adventures of Robin Hood, The (1938)',
 'Gate of Heavenly Peace, The (1995)',
 'Aladdin (1992)',
 'Of Love and Shadows (1994)',
 'Dances with Wolves (1990)',
 '2001: A Space Odyssey (1968)',
 'Secret Garden, The (1993)']

## 회고


### 1. 이번 프로젝트에서 어려웠던 점
 * Matrix Factorization(MF)
 * CSR 메트릭스 계산법이..   

### 2. 프로젝트를 진행하면서 알아낸 점 혹은 아직 모호한 점
* 알아낸 점 : 어떤 식으로 추천 및 비슷한 데이터를 불러오는지에 대한 전체적인 윤곽??

* 모호한 점 : 따라는 했으나, 해당 결과가 어떻게 이런 결과를 내는지는 모호함..



### 3. 루브릭 평가 지표를 맞추기 위해 시도한 것들.
 * 수업 예제와 비슷한(동일한) 내용 수행되도록 처리 하였다.


### 4. 만약에 루브릭 평가 관련 지표를 달성 하지 못했을 때, 이유에 관한 추정.
 * 내적 수치가 전혀 의미 있지 않았음( 이유는 잘 모르겠음 )
 * 아이템간 유사도, 기여도에 대한 부분이 의미 있는지 확인하는 과정이 없어보인다.

### 5. 자기 다짐
 * 다음 번엔 영화 제목을 그냥할게 아니고, 데이터 전처리( 대소문자 변환, 연도정보 삭제 등 )를 하고 진행하고 해야겠음 